In [5]:
from api_and_repo_link import link_me_up
import api_and_repo_link
from importlib import reload
reload(api_and_repo_link)
link_me_up()

C:\Users\Thomas.fuller\NotDrive\VENV\simplyvenv\Scripts\python.exe
C:\Users\Thomas.fuller\NotDrive\VENV\simplyvenv\Scripts\python.exe


In [9]:
from multithread_uploader import upload_all_these_records
from oliverbonas_source_dags.apis.api_netsuite import APIConfig_netsuite
api = APIConfig_netsuite()
from util.get_ns_number import get_ns_number
import pandas
StringCustomFieldRef = api.client.get_type(f"ns0:StringCustomFieldRef")
SalesOrder = api.client.get_type("ns19:SalesOrder")
get_ns_number(api, "SalesOrder")

failed for ns 0
failed for ns 1
failed for ns 2
failed for ns 3
failed for ns 4
failed for ns 5
failed for ns 6
failed for ns 7
failed for ns 8
failed for ns 9
failed for ns 10
failed for ns 11
failed for ns 12
failed for ns 13
failed for ns 14
failed for ns 15
failed for ns 16
failed for ns 17
failed for ns 18
SalesOrder = api.client.get_type("ns19:SalesOrder")


In [1]:
STATUSMAP = {
    "Fulfillmement": "_pendingFulfillment",
}

In [11]:

df = pandas.read_csv("SLS_NS_20220101_20230430_1.csv")

onerow = df.iloc[0].to_dict()

C:\Users\Thomas.fuller\NotDrive\VENV\simplyvenv\lib\site-packages\IPython\core\interactiveshell.py:3284 DtypeWarning: Columns (0,1,3,4,7,13,14,15,16,18,19,20,21,24,25,26,27,28,31,32,33,34,35,38) have mixed types.Specify dtype option on import or set low_memory=False.

{'ExternalID': '1/3462401/1',
 'tranID': '1/3462401/1',
 'Customer': 1512157.0,
 'trandate': '01/01/2022',
 'orderstatus': 'Fulfillmement',
 'Department': 20.0,
 'Class': 723.0,
 'Location': '2-WAREHOUSE',
 'discountrate': 0.0,
 'itemLine_item': 1454284.0,
 'itemLine_quantity': 1.0,
 'itemLine_salesPrice': 8.0,
 'itemLine_amount': 8.0,
 'itemLine_description': 'Inhale Exhale Mug',
 'itemLine_isTaxable': True,
 'itemLine_priceLevel': 'Markdown',
 'itemLine_CUSTOM_orderstatus': 'Complete',
 'itemLine_CUSTOM_itemLine_amount_net': 6.67,
 'itemLine_CUSTOM_Magento_Order_ID': '1003475850',
 'itemLine_CUSTOM_Order_date_time': '01/01/2022 00:00',
 'shipmethod': 'UK Standard Delivery - UK Standard Delivery',
 'shipaddressee': 'Claire Gilpin',
 'shipAddr1': nan,
 'shipAddr2': nan,
 'shipCity': 'Redhill',
 'shipState': 'Surrey',
 'shipZip': 'RH1 6PA',
 'shipCountry': 'GB',
 'billAddressee': 'Claire Gilpin',
 'billAddr1': nan,
 'billAddr2': nan,
 'billCity': 'Redhill',
 'billState': 'Surrey',
 'bil

In [36]:
import random
from datetime import datetime
orders = [SalesOrder(
    **{
        "class": {"name": onerow["Class"]},
        "department": {"name": onerow["Department"]},
        "externalId": onerow["ExternalID"] + "_"+str(random.randint(100, 100000)),
        "location": {"name": onerow["Location"]},
        "billingAddress": {
            "city": onerow["billCity"],
            #"country": onerow["billCountry"], # netsuite says not valid
            "state": onerow["billState"],
            "zip": onerow["billZip"],
            "addr1": onerow["billAddr1"],
            "addr2": onerow["billAddr2"],
            "addressee": onerow["billAddressee"],
        },
        # the error will be "please enter a value for entity" if the customer doesn't exist
       "entity": {"externalId": "customer_human_881003452"},#onerow["Customer"]},#881003451},
        "currency": {"name": onerow["currency"]},
        "discountRate": onerow["discountrate"],
        "email": onerow["email"],
        "exchangeRate": onerow["exchangerate"],
        # are we just using one sales order?
        "itemList": {
            "item": [
                {
                "item": {"externalId": str(int(onerow["itemLine_item"]))}, # ,"1594515"
                "amount": onerow["itemLine_amount"] * 10,
                "description": onerow["itemLine_description"],
                "isTaxable": onerow["itemLine_isTaxable"],
                #"priceLevel": {"name": onerow["itemLine_priceLevel"]}, # unexpected (check different wsdl)
                "quantity": onerow["itemLine_quantity"] * 10,
                #"salesPrice": onerow["itemLine_salesPrice"], # also unexpected
                #"line": 234256345,
                "options": [
                    # i can see that it works with those custom fields, but no others!
                    # they also don't show up in the UI.
                    # why only these columns?
                    ## seems like these need to be added!
                    {"customField": StringCustomFieldRef(
                        #Invalid options reference key Magento_Order_ID.
                        #Invalid options reference key null
                        internalId="4699",#"Magento_Order_ID",
                        value="2"#onerow["itemLine_CUSTOM_Magento_Order_ID"]
                    )},
                    {"customField": StringCustomFieldRef(
                        #internalId="Order_date_time",
                        internalId="1533",
                        value="firstname",
                        #value=onerow["itemLine_CUSTOM_Order_date_time"]
                    )},
                    """{"customField": stringcustomfieldref(
                        #internalId="itemLine_amount_net",
                        value=onerow["itemLine_CUSTOM_itemLine_amount_net"]
                    )},
                    {"customField": stringcustomfieldref(
                        #internalId="orderstatus",
                        value=onerow["itemLine_CUSTOM_orderstatus"]
                    )},"""
                    # when adding an item, the custom fields automatically pop up
                    # the only options are internalID, scriptID, best to speak to someone
                    #{"customField": {
                ]
                }
            ]
        },
        "orderStatus": STATUSMAP[onerow["orderstatus"]],
        "shippingAddress": {
            "city": onerow["shipCity"],
            #"country": onerow["shipCountry"], # netsuite says not valid
            "state": onerow["shipState"],
            "zip": onerow["shipZip"],
            "addr1": onerow["shipAddr1"],
            "addr2": onerow["shipAddr2"],
            "addressee": onerow["shipaddressee"],
        },
        "shipMethod": {"name": onerow["shipmethod"]},
        "taxRate": onerow["taxrate"],
        #'message': 'You do not have permissions to set a value for element tranid due to one of the following reasons: 1) The field is read-only; 2) An associated feature is disabled; 3) The field is available either when a record is created or updated, but not in both cases.'
        #"tranId": onerow["tranID"],
        # reformat date to netsuite friendly time
        "tranDate": datetime.strptime(onerow["trandate"], "%d/%m/%Y").strftime("%Y-%m-%dT%H:%M:%S.%fZ"),
       }
) for onerow in [row.to_dict() for pos, row in df.iterrows()][:1000]]

In [32]:
from multithread_uploader import upload_all_these_records
import multithread_uploader
reload(multithread_uploader)


<module 'multithread_uploader' from 'C:\\Users\\Thomas.fuller\\NotDrive\\Repos\\netsuite_uploader\\multithread_uploader.py'>

In [37]:
upload_all_these_records(orders, api, "addList")


thread (0) start_row: 0, end_row: 36
thread (1) start_row: 36, end_row: 72
thread (2) start_row: 72, end_row: 108
thread (3) start_row: 108, end_row: 144
thread (4) start_row: 144, end_row: 180
thread (5) start_row: 180, end_row: 216
thread (6) start_row: 216, end_row: 252
thread (7) start_row: 252, end_row: 288
thread (8) start_row: 288, end_row: 324
thread (9) start_row: 324, end_row: 360
thread (10) start_row: 360, end_row: 396
thread (11) start_row: 396, end_row: 432
thread (12) start_row: 432, end_row: 468
thread (13) start_row: 468, end_row: 504
thread (14) start_row: 504, end_row: 540
thread (15) start_row: 540, end_row: 576
thread (16) start_row: 576, end_row: 612
thread (17) start_row: 612, end_row: 648
thread (18) start_row: 648, end_row: 684
thread (19) start_row: 684, end_row: 720
thread (20) start_row: 720, end_row: 756
thread (21) start_row: 756, end_row: 792
thread (22) start_row: 792, end_row: 828
thread (23) start_row: 828, end_row: 864
thread (24) start_row: 864, end_

Exception in thread Thread-73:
Traceback (most recent call last):
  File "C:\Users\Thomas.fuller\AppData\Local\Programs\Python\Python39\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "C:\Users\Thomas.fuller\AppData\Local\Programs\Python\Python39\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Thomas.fuller\NotDrive\Repos\netsuite_uploader\multithread_uploader.py", line 33, in soap_uploader_thread
    response = api.service_proxy.addList(
  File "C:\Users\Thomas.fuller\NotDrive\VENV\simplyvenv\lib\site-packages\zeep\proxy.py", line 46, in __call__
    return self._proxy._binding.send(
  File "C:\Users\Thomas.fuller\NotDrive\VENV\simplyvenv\lib\site-packages\zeep\wsdl\bindings\soap.py", line 135, in send
    return self.process_reply(client, operation_obj, response)
  File "C:\Users\Thomas.fuller\NotDrive\VENV\simplyvenv\lib\site-packages\zeep\wsdl\bindings\soap.py", line 229, in process_reply


+++++++++ new thread (17) ++++++++++++++++++ new thread (18) +++++++++

+++++++++ new thread (19) +++++++++
+++++++++ new thread (20) +++++++++
+++++++++ new thread (21) +++++++++
+++++++++ new thread (22) +++++++++
+++++++++ new thread (23) +++++++++
+++++++++ new thread (24) +++++++++
+++++++++ new thread (25) +++++++++
+++++++++ new thread (26) +++++++++


    return self.process_error(doc, operation)
  File "C:\Users\Thomas.fuller\NotDrive\VENV\simplyvenv\lib\site-packages\zeep\wsdl\bindings\soap.py", line 329, in process_error
Exception in thread Thread-74:
Traceback (most recent call last):
  File "C:\Users\Thomas.fuller\AppData\Local\Programs\Python\Python39\lib\threading.py", line 980, in _bootstrap_inner
    raise Fault(
zeep.exceptions.Fault: java.lang.NumberFormatException: For input string: "nan"
    self.run()
  File "C:\Users\Thomas.fuller\AppData\Local\Programs\Python\Python39\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Thomas.fuller\NotDrive\Repos\netsuite_uploader\multithread_uploader.py", line 33, in soap_uploader_thread
    response = api.service_proxy.addList(
  File "C:\Users\Thomas.fuller\NotDrive\VENV\simplyvenv\lib\site-packages\zeep\proxy.py", line 46, in __call__
    return self._proxy._binding.send(
  File "C:\Users\Thomas.fuller\NotDrive\VENV\simplyvenv\lib\s

+++++++++ new thread (27) +++++++++
> maybe uploaded [1036] records
0:00:41.482825
+++++++++ thread completed (16) +++++++++
> maybe uploaded [1072] records
0:00:41.988376
+++++++++ thread completed (23) +++++++++
> maybe uploaded [1100] records
0:00:53.382941
+++++++++ thread completed (27) +++++++++
> maybe uploaded [1136] records
0:01:06.321648
+++++++++ thread completed (7) +++++++++
> maybe uploaded [1172] records
0:01:06.443856
+++++++++ thread completed (12) +++++++++
> maybe uploaded [1208] records
0:01:06.588584
+++++++++ thread completed (1) +++++++++
> maybe uploaded [1244] records
0:01:06.681798
+++++++++ thread completed (0) +++++++++
[2023-10-13 16:06:32,361] {connectionpool.py:322} WARNING - Connection pool is full, discarding connection: 1026328-sb1.suitetalk.api.netsuite.com. Connection pool size: 10
> maybe uploaded [1280] records
0:01:06.828069
[2023-10-13 16:06:32,381] {connectionpool.py:322} WARNING - Connection pool is full, discarding connection: 1026328-sb1.suit

In [38]:
import os
pandas.concat(
    [
        pandas.read_csv("thread_logs/" + filename)
        for filename in os.listdir("thread_logs")
    ]
)

,key_0,afterSubmitFailed,code,message_x,num_status_details,success,type,accountingBookDetailList,actualShipDate,altHandlingCost,...,total,totalCostEstimate,trackingNumbers,tranDate,tranId,tranIsVsoeBundle,validFrom,vatRegNum,vsoeAutoCalc,api_method
0,0,NaN,USER_ERROR,Nonexistent externalId 1454284 for item,1,False,ERROR,NaN,NaN,NaN,...,NaN,NaN,NaN,2022-01-01T00:00:00.000000Z,NaN,NaN,NaN,NaN,NaN,addList
1,1,NaN,USER_ERROR,Nonexistent externalId 1475203 for item,1,False,ERROR,NaN,NaN,NaN,...,NaN,NaN,NaN,2022-01-01T00:00:00.000000Z,NaN,NaN,NaN,NaN,NaN,addList
2,2,NaN,USER_ERROR,Nonexistent externalId 1503777 for item,1,False,ERROR,NaN,NaN,NaN,...,NaN,NaN,NaN,2022-01-01T00:00:00.000000Z,NaN,NaN,NaN,NaN,NaN,addList
3,3,NaN,USER_ERROR,Nonexistent externalId 1389050 for item,1,False,ERROR,NaN,NaN,NaN,...,NaN,NaN,NaN,2022-01-01T00:00:00.000000Z,NaN,NaN,NaN,NaN,NaN,addList
4,4,NaN,USER_ERROR,Nonexistent externalId 1389050 for item,1,False,ERROR,NaN,NaN,NaN,...,NaN,NaN,NaN,2022-01-01T00:00:00.000000Z,NaN,NaN,NaN,NaN,NaN,addList
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31,31,NaN,USER_ERROR,Nonexistent externalId 1126761 for item,1,False,ERROR,NaN,NaN,NaN,...,NaN,NaN,NaN,2022-01-03T00:00:00.000000Z,NaN,NaN,NaN,NaN,NaN,addList
32,32,NaN,USER_ERROR,Nonexistent externalId 1125856 for item,1,False,ERROR,NaN,NaN,NaN,...,NaN,NaN,NaN,2022-01-03T00:00:00.000000Z,NaN,NaN,NaN,NaN,NaN,addList
33,33,NaN,USER_ERROR,Nonexistent externalId 1398366 for item,1,False,ERROR,NaN,NaN,NaN,...,NaN,NaN,NaN,2022-01-03T00:00:00.000000Z,NaN,NaN,NaN,NaN,NaN,addList
34,34,NaN,USER_ERROR,Nonexistent externalId 1276091 for item,1,False,ERROR,NaN,NaN,NaN,...,NaN,NaN,NaN,2022-01-03T00:00:00.000000Z,NaN,NaN,NaN,NaN,NaN,addList
